In [1]:
import os
import pandas as pd
from pathlib import Path
# from get_all_trades import process_ca, process_data, updatePL
# from get_all_trades import calculate_PL
#from get_all_trades import convert_to_df


from functions import convert_to_df, process_ca,  process_data, process_depAndWith, process_div, process_tax, process_fees, get_all_trades, get_all_depAndWith, get_all_dividends, get_all_fees, calculate_PL, updatePL

In [2]:
cwd =  Path.cwd()

folder = cwd.parent /'Data'/'IBActivity'

tables = cwd.parent /'Data'/ 'Tables'
#cwd.parent / 'Data'
folder

WindowsPath('d:/Javier/SynologyDrive/Portfolio Python 2/Data/IBActivity')

In [3]:
files = list(folder.glob('*.csv'))


In [4]:
asset_categories_lst = ['Stocks','Equity and Index Options','Futures','Options On Futures', 'Forex']

In [5]:
a = 0
df = convert_to_df(files[a])
b = process_ca(df)

while  len(b) == 0:
    a = a+1
    df = convert_to_df(folder/files[a])
    b = process_ca(df)
        
print("File Found")
print (files[a])
print(b)

File Found
d:\Javier\SynologyDrive\Portfolio Python 2\Data\IBActivity\U9233567_20170612_20170612.csv
    Asset Category Currency             Date/Time Quantity Proceeds Symbol  \
133         Stocks      EUR  2017-06-09, 20:25:00     -600        0   POPD   

     Comm/Fee  T. Price  
133         0         0  


In [6]:
a = 0
df = convert_to_df(files[a])
b = process_depAndWith(df)

while  len(b) == 0:
    a = a+1
    df = convert_to_df(folder/files[a])
    b = process_depAndWith(df)
print("File Found")
print (files[a])
print(b)

File Found
d:\Javier\SynologyDrive\Portfolio Python 2\Data\IBActivity\U9233567_20161213_20161213.csv
        Date Currency Settle Date               Description Amount
0 2016-12-13      USD  2016-12-13  Electronic Fund Transfer  12000


In [7]:
a = 0
df = convert_to_df(files[a])
b = process_div(df)
while  len(b) == 0:
    a = a+1
    df = convert_to_df(folder/files[a])
    b = process_div(df)
print("File Found")
print (files[a])
print(b)

File Found
d:\Javier\SynologyDrive\Portfolio Python 2\Data\IBActivity\U9233567_20170501_20170501.csv
        Date Currency Symbol  Gross Amount
0 2017-05-01      USD     VZ         57.76


In [9]:
a = 0
df = convert_to_df(files[a])
b = process_tax(df)

while  len(b) == 0:
    a = a+1
    df = convert_to_df(folder/files[a])
    b = process_tax(df)

print("File Found")
print (files[a])
print(b)

File Found
d:\Javier\SynologyDrive\Portfolio Python 2\Data\IBActivity\U9233567_20170501_20170501.csv
        Date Currency Symbol  Tax Amount
0 2017-05-01      USD     VZ      -17.33


In [8]:
a = 0
df = convert_to_df(files[a])
b = process_fees(df)
while  len(b) == 0:
    a = a+1
    df = convert_to_df(folder/files[a])
    b = process_fees(df)
    
print("File Found")
print (files[a])
print(b)

File Found
d:\Javier\SynologyDrive\Portfolio Python 2\Data\IBActivity\U9233567_20170405_20170405.csv
       Subtitle Currency        Date  \
114  Other Fees      USD  2017-04-04   
115  Other Fees      USD  2017-04-04   

                                           Description Amount  
114                          AMEX Level I for Mar 2017   -1.5  
115  OPRA (US Options Exchanges) Non-Professional L...   -1.5  


In [9]:
for asset_category in asset_categories_lst:
    a = 0
    df = convert_to_df(files[a])
    b = process_data(df,asset_category)
    while  len(b) == 0:
        a = a+1
        df = convert_to_df(folder/files[a])
        b = process_data(df,asset_category)
    
    print("File Found")
    print (files[a])
    print(b)

File Found
d:\Javier\SynologyDrive\Portfolio Python 2\Data\IBActivity\U9233567_20170119_20170119.csv
   Asset Category Currency Symbol             Date/Time Quantity T. Price  \
45         Stocks      USD    GLD  2017-01-19, 10:56:05       20   114.14   

   Proceeds Comm/Fee  
45  -2282.8       -1  
File Found
d:\Javier\SynologyDrive\Portfolio Python 2\Data\IBActivity\U9233567_20170310_20170310.csv
              Asset Category Currency              Symbol  \
56  Equity and Index Options      USD  WFM 19MAY17 28.0 P   

               Date/Time Quantity T. Price Proceeds    Comm/Fee  
56  2017-03-10, 09:45:39       -1     0.86       86  -0.7569748  
File Found
d:\Javier\SynologyDrive\Portfolio Python 2\Data\IBActivity\U9233567_20171016_20171016.csv
    Asset Category Currency Symbol             Date/Time Quantity T. Price  \
171        Futures      USD   GEZ9  2017-10-13, 16:20:00        1   98.125   
172        Futures      USD   GEZ9  2017-10-13, 16:20:00        1    98.25   
173    

In [10]:
trades = get_all_dividends(folder)
trades.head()

FileNotFoundError: [Errno 2] File b'U9233567_20161125_20161125.csv' does not exist: b'U9233567_20161125_20161125.csv'

In [33]:
# def get_all_trades(folder):
#     #Creates a list of all files in folder
#     filelist = os.listdir(folder)
#     #Creates a empty list to append all dataframes
#     dataframe_list = []
#     #Iterates through all files in filelist
#     asset_categories_lst = ['Stocks','Equity and Index Options','Futures','Options On Futures', 'Forex']
#     for a_file in filelist:
#         #Checks if file is a .csv
#         if a_file.lower().endswith('.csv'):
#             data = convert_to_df(a_file)
#             for i in asset_categories_lst:
#                 df = process_data(data, i)
#                 dataframe_list.append(df)
#             dataframe_list.append(process_ca(data))
#     new_trades = pd.concat(dataframe_list,sort=False )
#     dataframe = pd.concat(dataframe_list,sort=False )
#     dataframe['Proceeds'] = dataframe['Proceeds'].astype('float')
#     dataframe['Quantity'] = dataframe['Quantity'].astype('float')
#     dataframe['T. Price'] = dataframe['T. Price'].astype('float')
#     dataframe['Comm/Fee'] = dataframe['Comm/Fee'].astype('float')
#     dataframe['Date/Time'] = pd.to_datetime(dataframe['Date/Time'])
#     dataframe['Quantity_Rsum'] =  dataframe.groupby(['Symbol'])['Quantity'].transform(lambda x : x.cumsum())
#     return dataframe

In [34]:
# def convert_to_df(a_file):
#     data = pd.read_csv(folder/a_file, names=list('abcdefghijklmnopq'))
#     #data.drop(data.loc[(data.a == 'Notes/Legal Notes') | (data.a == 'Codes') ].index, inplace= True)
#     #data = (data.a != 'Codes') & (data.a != 'Notes/Legal Notes')
#     data = data.loc[~data.a.isin(['Statement','Codes', 'Notes/Legal Notes','Account Information','Change in NAV','Mark-to-Market Performance Summary','Realized & Unrealized Performance Summary','Open Positions'])]
#     return data

In [35]:
# def calculatePL (df):
#     newdf = df.loc[:,['Quantity','Quantity_Rsum','T. Price','Proceeds']]
#     newdf['prev'] = newdf['Quantity_Rsum'].shift(1 , fill_value=0)
#     #Make a new column OIDC
#     oidc_options = [
#         newdf['prev'] == 0,
#         newdf['Quantity_Rsum'] == 0,
#         newdf['prev']/newdf['Quantity_Rsum'] < 0,
#         abs(newdf['Quantity_Rsum']) > abs(newdf['prev'])
#     ]
#     oidc_choices = [
#         'Open','Close','Reversal', 'Increase'
#     ]
#     newdf['oidc'] = np.select(oidc_options, oidc_choices, default = 'Decrease')

#     newdf['index'] = np.where((newdf['oidc']== 'Open')|(newdf['oidc']=='Reversal'),1,0)

#     newdf['Increase'] = np.where((newdf['oidc'] == 'Open') | (newdf['oidc'] == 'Increase') | (newdf['oidc'] =='Reversal'),1,0)
#     newdf['ProceedsIncrease'] = newdf['Proceeds']*newdf['Increase']
#     newdf['QuantityIncrease'] = newdf['Quantity']*newdf['Increase']
#     newdf['cumsum'] = newdf['index'].cumsum()
#     newdf['TotProceedsIncrease']= newdf.groupby(['cumsum'])['ProceedsIncrease'].cumsum()
#     newdf['TotQuantityIncrease']= newdf.groupby(['cumsum'])['QuantityIncrease'].cumsum()
#     newdf['AvgOpenPrice'] = newdf['TotProceedsIncrease'] / newdf['TotQuantityIncrease'] * -1
#     newdf['AvgOpenPriceBefore'] = newdf['AvgOpenPrice'].shift(1 , fill_value=0)

#     plOptions = [
#         newdf['oidc'] == 'Open',
#         newdf['oidc'] == 'Increase',
#         newdf['oidc'] == 'Decrease',
#         newdf['oidc'] == 'Close',
#         newdf['oidc'] == 'Reversal'
#     ]
#     plChoices = [
#         # Open case P/L = 0
#         0,
#         #Increase case 
#         0,
#         #Decrease case: Average Open Price - T. Price * Quantity (* Multiplier +coommfee)
#         (newdf['AvgOpenPriceBefore'] - newdf['T. Price']) * newdf['Quantity'],
#         #Close case 
#         (newdf['AvgOpenPriceBefore'] - newdf['T. Price']) * newdf['Quantity'],
#         #Reverse case
#         (newdf['AvgOpenPriceBefore'] - newdf['T. Price']) * newdf['Quantity']
#     ]
#     newdf['PL'] = np.select(plOptions, plChoices, default = 0)
#     newdf['CumPL'] = newdf['PL'].cumsum()
#     newdf.drop(['Quantity',	'Quantity_Rsum', 'T. Price', 'Proceeds', 'prev', 'oidc', 'index','Increase','ProceedsIncrease', 'QuantityIncrease', 'cumsum', 'TotProceedsIncrease','TotQuantityIncrease','AvgOpenPriceBefore'], axis=1, inplace=True)
#     #return newdf['PL'], newdf['AvgOpenPrice'], newdf['CumPL']
#     return newdf

# def updatePL (df):
#     df.drop(['AvgOpenPrice','PL','CumPL'], axis=1, inplace = True, errors= 'ignore')
#     dfgrouped = df.groupby('Symbol')
#     dataframe_list = []
#     for name, group in dfgrouped:
#         symbol_group = dfgrouped.get_group(name)
#         profLoss = calculatePL( symbol_group)
#         symbol_group = symbol_group.join(profLoss)
#         dataframe_list.append(symbol_group)
#     new_trades = pd.concat(dataframe_list,sort=False ).sort_index()
#     return new_trades
